In [1]:
import json
import re
import nltk
import emoji
import numpy as nps
import pandas as pd
from unicodedata import normalize
from wordcloud import WordCloud
from pysentimiento.preprocessing import preprocess_tweet

## Dataset: SENT-COVID

In [2]:
with open('data/SENT-COVID.json') as file:
    data = json.load(file)
    
pd.options.mode.chained_assignment = None                                         
pd.set_option('display.max_colwidth',None)   

df = pd.DataFrame(data)
df[['Tweet','clean_tweet','Label']].head(10)

,Tweet,clean_tweet,Label
0,1401047081121353728-@dulcema201 @BronstonRaqsa02 Protocolo de COVID !!!!,Protocolo de COVID !!!!,NEUTRO
1,1258159310162595843-#COVID19 #QuedateEnCasa en Morelia Centro,COVID QuedateEnCasa en Morelia Centro,NEUTRO
2,1272748988626862082-México va en en aumento con el #Covid_19. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener 120 días en paro total. Pero podemos ser precavidos al usar la #SanaDistancia,México va en en aumento con el Covid. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener días en paro total. Pero podemos ser precavidos al usar la SanaDistancia,POSITIVO
3,1349385638722883585-@sororavirus Creo en todo y nada. 💜,Creo en todo y nada. 💜,NEUTRO
4,1360615587114844161-@GobiernoMX había prometido 389,había prometido,NEGATIVO
5,1384487745033162755-Vacunación en Tonalá,Vacunación en Tonalá,POSITIVO
6,1335343584875327494-No es cierto lo que dice @WHO,No es cierto lo que dice,NEGATIVO
7,1257755579101003776-Participa en familia con esta entretenida actividad #MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.#aquiestamos #IED #QuedateEnCasa https://t.co/BQZTpxwca2,Participa en familia con esta entretenida actividad MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.aquiestamos IED QuedateEnCasa,POSITIVO
8,1317288163065401345-@lalitogonzaalez Acabar con el Coronavirus.,Acabar con el Coronavirus.,POSITIVO
9,1255373406624776194-#InternationalDanceDay #DID #ENCASA https://t.co/crIQOdoPgG,InternationalDanceDay DID ENCASA,NEUTRO


## Limpieza y Normalización

In [3]:
def remove_pattern(text, pattern):
    r = re.findall(pattern, text)
    for i in r:
        text = re.sub(i, '', text)
    return text

def clean_text(text):
  text = re.sub(r'^RT[\s]+', '', text) # RT's
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text) # Url's
  text = re.sub(r'-', '', text) #guiones
  text = re.sub(r'_', '', text) #guiones bajos 
  text = re.sub(r'@[A-Za-z0-9_]+', '', text) #menciones
  text = re.sub(r'[~^0-9]', '', text) #numeros
  text = re.sub("\\s+", ' ', text) ##Espacios blancos dobles
  text = re.sub('\n', ' ', text) ##Saltos de linea

  pattern = r'([.])([A-Z])'
  text = re.sub(pattern, r'\1 \2', text) # Separacion de punto seguido por una mayuscula
  return text

def normalize(text):    
   text = text.lower() # Se convierte todo el texto a minúsculas
   text = re.sub(r'#', '', text) # Eliminacion de almohadillas 
   text = re.sub('http\S+', ' ', text) # Eliminación de páginas web 
   text = re.sub('emoji', '', text) # Eliminación de palabra emoji
   regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
   text = re.sub(regex , ' ', text) # Eliminacion de signos de puntuacion
   text = re.sub("\d+", ' ', text) # Eliminación de números
   text = re.sub("\\s+", ' ', text) # Eliminación de espacios en blanco múltiples
   #text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD"), 0, re.I)
   #text = normalize( 'NFC')
   return(text)

def tokenize(text):    
    text= text.split(sep = ' ')  # Tokenización por palabras individuales
    text= [token for token in text if len(token) > 1]  # Eliminación de tokens con una longitud < 2
    return(text)

def remove_punct(text):
 text = "".join(u for u in text if u not in ("?","¿", ".", ";", ":", "!","¡",'"',"%","“","”","$","&","'","\\",
                                                "*","+",",","/","<",">","=","^","•","...", "ç","π","ⓘ"))
 a,b = 'áéíóúÁÉÍÓÚ','aeiouAEIOU'
 trans = str.maketrans(a,b)     
 text = text.translate(trans) # Reemplazo de palabras acentuadas                                    
 return text

def remove_emoji(text):
  return emoji.get_emoji_regexp().sub(r'', text) # Eliminacion de emojis

def py_preprocess(text):
  return preprocess_tweet(text)  # Preprocesamiento de pysentimiento (opcional)

def split_hashtags(text):
  pat1 = r'([#])'
  text = re.sub(pat1, ' ', text) # Reemplazo de almohadillas por espacios en blanco
  pat2 = r'([a-z])([A-Z])'
  text = re.sub(pat2, r'\1 \2', text) # Separacion de minisculas seguidas por una mayuscula
  return text    


In [4]:
df['clean_tweet'] = df['Tweet'].apply(clean_text)
df['nopunct_tweet'] = df['clean_tweet'].apply(remove_punct)
df['noemoji_tweet'] = df['nopunct_tweet'].apply(remove_emoji)
df['nohash_tweet'] = df['noemoji_tweet'].apply(split_hashtags)
df['norm_tweet'] = df['nohash_tweet'].apply(normalize)
df['tokenized_tweet'] = df['norm_tweet'].apply(tokenize)
df[['Label', 'clean_tweet', 'norm_tweet','tokenized_tweet']][120:150]

,Label,clean_tweet,norm_tweet,tokenized_tweet
120,POSITIVO,Jajaja algo bien común que veo en la consulta... la mamá diciéndole al Doc todos los síntomas de su hijo adulto 🤦‍♀️ #DíadelasMadres,jajaja algo bien comun que veo en la consulta la mama diciendole al doc todos los sintomas de su hijo adulto diadelas madres,"[jajaja, algo, bien, comun, que, veo, en, la, consulta, la, mama, diciendole, al, doc, todos, los, sintomas, de, su, hijo, adulto, diadelas, madres]"
121,NEUTRO,Perdón mi ignorancia pero ese señor no fue el primero que le dio covid “conocido” de nuestro país? No se supone que se fue a esquiar a Vail y regreso enfermo? Pregunto por qué si es así ¿cuánto tiempo duro su lucha por la enfermedad?,perdon mi ignorancia pero ese señor no fue el primero que le dio covid conocido de nuestro pais no se supone que se fue a esquiar a vail y regreso enfermo pregunto por que si es asi cuanto tiempo duro su lucha por la enfermedad,"[perdon, mi, ignorancia, pero, ese, señor, no, fue, el, primero, que, le, dio, covid, conocido, de, nuestro, pais, no, se, supone, que, se, fue, esquiar, vail, regreso, enfermo, pregunto, por, que, si, es, asi, cuanto, tiempo, duro, su, lucha, por, la, enfermedad]"
122,POSITIVO,#MédicosDelBienestar: Convocatoria especial para la atención del #COVID. ¡Inscríbete ya! ➡️,medicos del bienestar convocatoria especial para la atencion del covid inscribete ya,"[medicos, del, bienestar, convocatoria, especial, para, la, atencion, del, covid, inscribete, ya]"
123,NEGATIVO,"""😔 México reporta más de mil casos positivos de COVID por tercer día consecutivo.",mexico reporta mas de mil casos positivos de covid por tercer dia consecutivo,"[mexico, reporta, mas, de, mil, casos, positivos, de, covid, por, tercer, dia, consecutivo]"
124,NEGATIVO,El es una mierda igual que su jefe . Porque lo escribo??Niegan medicamentos. Dan recetas incompletas a pacientes diabéticos. Nunca hay médicos en los consultorios. Exponen a los pacientes al #coronavirus al tenerlos amontonados en la #Unifila,el es una mierda igual que su jefe porque lo escribo niegan medicamentos dan recetas incompletas a pacientes diabeticos nunca hay medicos en los consultorios exponen a los pacientes al coronavirus al tenerlos amontonados en la unifila,"[el, es, una, mierda, igual, que, su, jefe, porque, lo, escribo, niegan, medicamentos, dan, recetas, incompletas, pacientes, diabeticos, nunca, hay, medicos, en, los, consultorios, exponen, los, pacientes, al, coronavirus, al, tenerlos, amontonados, en, la, unifila]"
125,POSITIVO,Siempre que estoy en crisis o en problemas olvido que es el Sagrado Corazón de Jesús quien me socorre y defiende. Él tiene siempre el modo o la manera precisa de recordarme que nunca me abandonará,siempre que estoy en crisis o en problemas olvido que es el sagrado corazon de jesus quien me socorre y defiende el tiene siempre el modo o la manera precisa de recordarme que nunca me abandonara,"[siempre, que, estoy, en, crisis, en, problemas, olvido, que, es, el, sagrado, corazon, de, jesus, quien, me, socorre, defiende, el, tiene, siempre, el, modo, la, manera, precisa, de, recordarme, que, nunca, me, abandonara]"
126,NEGATIVO,Ackerman no puede ser rector porque no nació en México. Aunque no me extrañaría su plan de López para desestabilizar a la UNAM. No pudo tumbar a Graue antes de la pandemia,ackerman no puede ser rector porque no nacio en mexico aunque no me extrañaria su plan de lopez para desestabilizar a la unam no pudo tumbar a graue antes de la pandemia,"[ackerman, no, puede, ser, rector, porque, no, nacio, en, mexico, aunque, no, me, extrañaria, su, plan, de, lopez, para, desestabilizar, la, unam, no, pudo, tumbar, graue, antes, de, la, pandemia]"
127,NEGATIVO,El pinche covid es un perro virus traicionero.,el pinche covid es un perro virus traicionero,"[el, pinche, covid, es, un, perro, virus, traicionero]"
128,NEUTRO,#cuandoseamosgatos #sinfiltro #fotodeldia #encierro #cuarentena #ciudaddemexico #estotambienescdmx #amanecer en 

## Lemmatización 

In [ ]:
import spacy
from nltk.stem.snowball import SnowballStemmer

sp = spacy.load('es_core_news_sm')

def lemmatization(text):
    doc = sp(text)
    return ' '.join([word.lemma_ for word in doc]) 

#stemmer = SnowballStemmer('spanish')
#stemmed_spanish = [stemmer.stem(item) for item in spanish_words]


In [ ]:
df['lem_tweet'] = df['norm_tweet'].apply(lemmatization)
df['lem_tweet'] = df['lem_tweet'].apply(remove_punct)
df['lemtokenized_tweet'] = df['lem_tweet'].apply(tokenize)
df[['Label', 'norm_tweet','lem_tweet','tokenized_tweet','lemtokenized_tweet']][90:120]

,Label,norm_tweet,lem_tweet,tokenized_tweet,lemtokenized_tweet
90,NEGATIVO,seguimos con las medidas por etapa critica covid,seguir con el medida por etapa critica covid,"[seguimos, con, las, medidas, por, etapa, critica, covid]","[seguir, con, el, medida, por, etapa, critica, covid]"
91,NEGATIVO,que tal que yo me enfermo y muero que tal si fallece mi familia me estoy volviendo loca… me voy a morir el deterioro mental que padece el personal sanitario del pais en el combate al coronavirus covid,que tal que yo yo enfermir y muero que tal si fallecer mi familia yo estar volver loca … yo ir a morir el deterioro mental que padecer el personal sanitario del pais en el combate al coronavirus covid,"[que, tal, que, yo, me, enfermo, muero, que, tal, si, fallece, mi, familia, me, estoy, volviendo, loca…, me, voy, morir, el, deterioro, mental, que, padece, el, personal, sanitario, del, pais, en, el, combate, al, coronavirus, covid]","[que, tal, que, yo, yo, enfermir, muero, que, tal, si, fallecer, mi, familia, yo, estar, volver, loca, yo, ir, morir, el, deterioro, mental, que, padecer, el, personal, sanitario, del, pais, en, el, combate, al, coronavirus, covid]"
92,POSITIVO,que los penes se multipliquen como el covid y se le peguen en la garganta jajja,que el pen el multiplicar como el covid y el el pegar en el garganta jajja,"[que, los, penes, se, multipliquen, como, el, covid, se, le, peguen, en, la, garganta, jajja]","[que, el, pen, el, multiplicar, como, el, covid, el, el, pegar, en, el, garganta, jajja]"
93,NEGATIVO,se disparan indices criminales en nl de acuerdo a cifras oficiales en nuevo leon,el disparar indiz criminal en nl de acuerdo a cifra oficial en nuevo leon,"[se, disparan, indices, criminales, en, nl, de, acuerdo, cifras, oficiales, en, nuevo, leon]","[el, disparar, indiz, criminal, en, nl, de, acuerdo, cifra, oficial, en, nuevo, leon]"
94,POSITIVO,todos somos uno nos vemos en la escuela ante el confinamiento por covid los maestros han demostrado que nada los detiene al innovar en sus clases en lineak hoy el te invita a contar tu experiencia convocatoria,todo ser uno yo ver en el escuela ante el confinamiento por covid el maestro haber demostrar que nada el detener al innovar en su clase en lineak hoy el tu invitar a contar tu experiencia convocatorio,"[todos, somos, uno, nos, vemos, en, la, escuela, ante, el, confinamiento, por, covid, los, maestros, han, demostrado, que, nada, los, detiene, al, innovar, en, sus, clases, en, lineak, hoy, el, te, invita, contar, tu, experiencia, convocatoria]","[todo, ser, uno, yo, ver, en, el, escuela, ante, el, confinamiento, por, covid, el, maestro, haber, demostrar, que, nada, el, detener, al, innovar, en, su, clase, en, lineak, hoy, el, tu, invitar, contar, tu, experiencia, convocatorio]"
95,NEGATIVO,sntesalud alerta mexico vive alto contagio coronavirus covid ante sintomas no te automediques ni tomes farmacos que prometen curar el covid y llama al quedate en casa feliz viernes de mayo mexico seccion feliz finde rt,sntesalud alerta mexico vivir alto contagio coronavirus covid ante sintomas no tu automediques ni tom farmaco que prometar curar el covid y llamar al quedate en casa feliz viernes de mayo mexico seccion feliz findir rt,"[sntesalud, alerta, mexico, vive, alto, contagio, coronavirus, covid, ante, sintomas, no, te, automediques, ni, tomes, farmacos, que, prometen, curar, el, covid, llama, al, quedate, en, casa, feliz, viernes, de, mayo, mexico, seccion, feliz, finde, rt]","[sntesalud, alerta, mexico, vivir, alto, contagio, coronavirus, covid, ante, sintomas, no, tu, automediques, ni, tom, farmaco, que, prometar, curar, el, covid, llamar, al, quedate, en, casa, feliz, viernes, de, mayo, mexico, seccion, feliz, findir, rt]"
96,NEGATIVO,pues como foca aplaudidora te queda bien el papel,pues como foco aplaudidoro tu quedar bien el papel,"[pues, como, foca, aplaudidora, te, queda, bien, el, papel]","[pues, como, foco, aplaudidoro, tu, quedar, bien, el, papel]"
97,NEGATIVO,no habra 

## Nltk Stopwords

In [ ]:
from nltk.corpus import stopwords

# Obtención de listado de stopwords del español
stop_words_esp = list(stopwords.words('spanish'))

def remove_stopwords(text):
    text = [w for w in text if not w in stop_words_esp]
    return text

df['lemtoksw_tweet'] = df['lemtokenized_tweet'].apply(remove_stopwords)
df['normtoksw_tweet'] = df['tokenized_tweet'].apply(remove_stopwords)

print(stop_words_esp[:100])


['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo']


## Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X1 = df['norm_tweet'] #Tweets normalizados
X2 = df['lem_tweet'] #Tweets lemmatizados
y = df['Label'] #Etiquetas

# Separamos el corupus 1) con columna 'norm_tweet' 2) con columna 'lem_tweet'
#X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.25 ,random_state=37)
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25 ,random_state=37)

In [ ]:
value, counts = np.unique(y_train, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))
value, counts = np.unique(y_test, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))

{'NEGATIVO': 33.7124526100904, 'NEUTRO': 44.911052785068534, 'POSITIVO': 21.37649460484106}
{'NEGATIVO': 35.345581802274715, 'NEUTRO': 43.39457567804025, 'POSITIVO': 21.25984251968504}


## Vectorizaciones

### TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf= TfidfVectorizer(min_df= 3, ngram_range=(1,2), stop_words = stop_words_esp).fit(X_train)
                        
print('Numero de features: ' +str(len(tfidf.get_feature_names_out())))
tfidf.fit(X_train)

Numero de features: 2291


TfidfVectorizer(min_df=3, ngram_range=(1, 2),
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...])

In [ ]:
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)
X_train_tfidf 

<3429x2291 sparse matrix of type '<class 'numpy.float64'>'
	with 23705 stored elements in Compressed Sparse Row format>

### CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

countvect = CountVectorizer(min_df=3, stop_words = stop_words_esp, ngram_range=(1,2)).fit(X_train)

print('Numero de features: ' +str(len(countvect.get_feature_names_out())))
countvect.fit(X_train)

Numero de features: 2291


CountVectorizer(min_df=3, ngram_range=(1, 2),
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...])

In [ ]:
# Creacion de la matrices
X_train_cv = countvect.transform(X_train)
X_test_cv  = countvect.transform(X_test)
X_train_cv

<3429x2291 sparse matrix of type '<class 'numpy.int64'>'
	with 23705 stored elements in Compressed Sparse Row format>

### Word2Vec

In [ ]:
from gensim.models import Word2Vec

X3 = df['normtoksw_tweet'] # Tweet normalizado, tokenizado y sin stopwotds
X4 = df['lemtoksw_tweet'] # Tweet lemmatizado, tokenizado y sin stopwotds

embedding=200
w2v = Word2Vec(X4, min_count=3, vector_size=embedding, window=5, sg=1 )
w2v.train(X4, total_examples= len(df['lem_tweet']), epochs=20)

#w2v.wv.most_similar('idiota')

(497811, 759520)

In [ ]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

wordvec_arrays = np.zeros((len(X3), 200)) 
for i in range(len(X3)):
    wordvec_arrays[i,:] = word_vector(X3[i], 200)
    
X_w2v = pd.DataFrame(wordvec_arrays)
X_w2v.shape

(4572, 200)

### Vocabulario 

In [ ]:
# vocabuilario TFIDF
#print(f" Número de tokens creados: {len(tfidf.get_feature_names_out())}")
#tfidf.get_feature_names()

# vocabulario CountVectorizer
print(f" Número de tokens creados: {len(countvect.get_feature_names_out())}")
countvect.get_feature_names_out()

 Número de tokens creados: 2291


array(['aaa', 'abierto', 'abrazo', ..., 'ılılı', 'ılılı aire',
       'ılılı conferencia'], dtype=object)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

value, counts = np.unique(y_test, return_counts=True)
pesos=dict(zip(value, 100 * counts / sum(counts))) # Proporcion de cada clase en el corpus

lr = LogisticRegression(C=1, solver='lbfgs', penalty='l2', class_weight=None, multi_class='ovr', random_state=2, max_iter=1000)
lr.fit(X_w2v, y)

predic_lr = lr.predict(X_w2v)
probas_lr = lr.predict_proba(X_w2v)
scores_lr = lr.decision_function(X_w2v)

In [ ]:
print(lr.score(X_w2v, y))
print(lr.score(X_w2v, y))
print(lr.classes_)

score_list = list(zip(y_test[0:5],scores_lr[0:5]))
proba_list = list(zip(y_test[0:5],probas_lr[0:5]))



0.5507436570428696
0.5507436570428696
['NEGATIVO' 'NEUTRO' 'POSITIVO']


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve


lr_auc = roc_auc_score(y, probas_lr, multi_class = 'ovr')
print(lr_auc)


0.7183152720773743


In [ ]:
%matplotlib inline

fig = plt.figure()
x=np.linspace(1,1143,1143)
plt.plot(x, y_test)

In [ ]:
fature_names = np.array(countvect.get_feature_names_out())

sorted_coef_index = lr.coef_[0].argsort() # Coeficientes ordenados clase NEGATIVO
#sorted_coef_index = lr.coef_[1].argsort() # Coeficientes ordenados clase NEUTRO
#sorted_coef_index = lr.coef_[2].argsort() # Coeficientes ordenados clase POSITIVO

sorted_coef_index2 = X_train_cv.max(0).toarray()[0].argsort()

print("Coeficientes (clase negativo) mas pequeños del modelo:\n{}\n".format(fature_names[sorted_coef_index[:20]]))
print("Coeficientes (clase negativo) mas grandes del modelo:\n{}\n".format(fature_names[sorted_coef_index[:-20:-1]]))
print("Coeficientes tfidf mas pequeños:\n{}\n".format(fature_names[sorted_coef_index2[:20]]))
print("Coeficientes tfidf mas grandes:\n{}\n".format(fature_names[sorted_coef_index2[:-20:-1]]))

Coeficientes (clase negativo) mas pequeños del modelo:
['casa' 'cuarentenar' 'feliz' 'excelente' 'gracias' 'compartir'
 'contingencia' 'vacuna covid' 'hoy' 'año' 'quedate' 'efecto' 'hola' 'dr'
 'quedatir' 'recomendacion' 'pfizer' 'amor' 'quedate casa' 'cdmx']

Coeficientes (clase negativo) mas grandes del modelo:
['morir' 'maldito' 'mal' 'covidiota' 'muerte' 'pendejo' 'caso' 'pinche'
 'peor' 'mil' 'valer' 'poblacion' 'mentira' 'millon' 'defuncion' 'madre'
 'gente' 'dejar' 'gobierno']

Coeficientes tfidf mas pequeños:
['aaa' 'papel' 'papas' 'papa' 'panorama' 'panico' 'panel' 'pandemio'
 'pandemiar' 'pandemia ser' 'pandemia mundial' 'pandemia mas'
 'pandemia ir' 'pandemia haber' 'paquete' 'pandemia covid' 'pan' 'palabra'
 'pais' 'pai']

Coeficientes tfidf mas grandes:
['mexico' 'moren' 'regreso' 'haber' 'ser' 'malo' 'sano' 'querer' 'gatell'
 'arte' 'maestro' 'tras' 'esperar' 'perder' 'dosis' 'casa' 'silla'
 'vacuna' 'dia']



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy: ' + str(accuracy_score(y, predic_lr)))
print('Precision: ' + str(precision_score(y, predic_lr, average='weighted')))
print('Recall: ' + str(recall_score(y, predic_lr, average='weighted')))
print('f1-score: ' + str(f1_score(y, predic_lr, average='weighted')))


Accuracy: 0.5507436570428696
Precision: 0.5560251596744563
Recall: 0.5507436570428696
f1-score: 0.530904646154421


In [ ]:
from sklearn.metrics import classification_report

new = np.asarray(y_test)
print(classification_report(y, predic_lr))

              precision    recall  f1-score   support

    NEGATIVO       0.57      0.55      0.56      1560
      NEUTRO       0.54      0.68      0.60      2036
    POSITIVO       0.56      0.28      0.37       976

    accuracy                           0.55      4572
   macro avg       0.56      0.50      0.51      4572
weighted avg       0.56      0.55      0.54      4572



In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


df_matrix = pd.DataFrame(confusion_matrix(y_test, predic_lr))
df_matrix
#sns.heatmap(df_matrix, annot=True, vmin=0, vmax=400)
#plt.figure(figsize=(5,4))
#plt.ylabel()


,0,1,2
0,225,151,28
1,76,375,45
2,32,118,93


## Validacion cruzada

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score, validation_curve

tfidf2 = TfidfVectorizer(min_df=3, stop_words = stop_words_esp, ngram_range=(1,2)).fit(X2)
X_tfidf = tfidf2.transform(X2)
lr.fit(X_tfidf, y)

#countvect2 = CountVectorizer(min_df=3, stop_words = stop_words_esp, ngram_range=(1,2)).fit(X2)
#X_cv = countvect2.transform(X2)
#lr.fit(X_cv,y)

cv_scores = cross_val_score(lr,X_w2v,y, cv=50)
predicted_labels = model_selection.cross_val_predict(lr,X_w2v, y, cv = 10)

print(cv_scores.mean())
print(cv_scores.max(),cv_scores.min())


0.537651696129957
0.6413043478260869 0.42391304347826086


In [ ]:
# Curva de valicacion 
param_range =np.linspace(-10,10)
train_scores, test_scores = validation_curve(lr, X2_cv, y, param_name='C', param_range=param_range, cv=10)

## Optimizacion de hyperparametros

### GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=37)
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]

search_g = GridSearchCV(lr, space, cv=cv, scoring='accuracy', n_jobs=-1)
result1 = search_g.fit(X_train_cv, y_train)


In [ ]:
print('Best Score: %s' % result1.best_score_)
print('Best Hyperparameters: %s' % result1.best_params_)

Best Score: 0.572275643757921
Best Hyperparameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}


### RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=37)
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = loguniform(1e-5, 100)

search = RandomizedSearchCV(lr, space, n_iter=100, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)
result = search.fit(X_train_cv, y_train)


In [ ]:
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.5810251251712046
Best Hyperparameters: {'C': 0.30971587230022724, 'penalty': 'l2', 'solver': 'lbfgs'}
